# 基于回归分析的大学综合得分预测
---

## 一、案例简介
大学排名是一个非常重要同时也极富挑战性与争议性的问题，一所大学的综合实力涉及科研、师资、学生等方方面面。目前全球有上百家评估机构会评估大学的综合得分进行排序，而这些机构的打分也往往并不一致。在这些评分机构中，世界大学排名中心（Center for World University Rankings，缩写CWUR）以评估教育质量、校友就业、研究成果和引用，而非依赖于调查和大学所提交的数据著称，是非常有影响力的一个。

本任务中我们将根据 CWUR 所提供的世界各地知名大学各方面的排名（师资、科研等），一方面通过数据可视化的方式观察不同大学的特点，另一方面希望构建机器学习模型（线性回归）预测一所大学的综合得分。

## 二、数据概览
假设数据文件位于当前文件夹，我们用 pandas 读入标准 csv 格式文件的函数`read_csv()`将数据转换为`DataFrame`的形式。观察前几条数据记录：

In [1]:
import pandas as pd
import numpy as np

data_df = pd.read_csv('./data/cwurData.csv')  # 读入 csv 文件为 pandas 的 DataFrame
data_df.head(3).T  # 观察前几列并转置方便观察

,0,1,2
world_rank,1,2,3
institution,Harvard University,Massachusetts Institute of Technology,Stanford University
region,USA,USA,USA
national_rank,1,2,3
quality_of_education,7,9,17
alumni_employment,9,17,11
quality_of_faculty,1,3,5
publications,1,12,4
influence,1,4,2
citations,1,4,2


In [2]:
data_df = data_df.dropna()  # 舍去包含 NaN 的 row
len(data_df)
data_df.shape

(2000, 14)

In [3]:
feature_cols = ['quality_of_faculty', 'publications', 'citations', 'alumni_employment', 
                'influence', 'quality_of_education', 'broad_impact', 'patents']
X = data_df[feature_cols]
Y = data_df['score']
X.T

,200,201,202,203,204,205,206,207,208,209,...,2190,2191,2192,2193,2194,2195,2196,2197,2198,2199
quality_of_faculty,1.0,4.0,2.0,5.0,10.0,9.0,6.0,8.0,3.0,11.0,...,218.0,218.0,218.0,218.0,218.0,218.0,218.0,218.0,218.0,218.0
publications,1.0,5.0,15.0,10.0,11.0,14.0,7.0,17.0,70.0,18.0,...,830.0,962.0,937.0,811.0,595.0,926.0,997.0,830.0,886.0,861.0
citations,1.0,3.0,2.0,12.0,11.0,9.0,3.0,10.0,19.0,32.0,...,812.0,645.0,812.0,511.0,645.0,812.0,645.0,812.0,812.0,812.0
alumni_employment,1.0,2.0,11.0,10.0,12.0,8.0,22.0,14.0,16.0,25.0,...,542.0,540.0,327.0,567.0,449.0,567.0,566.0,549.0,567.0,567.0
influence,1.0,3.0,2.0,9.0,12.0,13.0,4.0,19.0,25.0,7.0,...,974.0,865.0,962.0,969.0,430.0,845.0,908.0,823.0,974.0,991.0
quality_of_education,1.0,11.0,3.0,2.0,7.0,13.0,4.0,10.0,5.0,9.0,...,367.0,367.0,367.0,367.0,367.0,367.0,236.0,367.0,367.0,367.0
broad_impact,1.0,4.0,2.0,13.0,12.0,13.0,7.0,18.0,41.0,19.0,...,984.0,969.0,998.0,975.0,994.0,969.0,981.0,975.0,975.0,981.0
patents,2.0,6.0,1.0,48.0,16.0,4.0,28.0,149.0,204.0,45.0,...,434.0,774.0,861.0,756.0,839.0,816.0,871.0,824.0,651.0,547.0


In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

In [5]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.2,random_state=250)
Y.shape, X.shape, x_train.shape, x_test.shape, y_train.shape, y_test.shape # 输出数据行列信息

((2000,), (2000, 8), (1600, 8), (400, 8), (1600,), (400,))

## 三、模型构建

### 多元线性回归sklearn实现

In [6]:
linear = LinearRegression()
linear.fit(x_train, y_train)
yp=linear.predict(x_test)
mse=MSE(yp,y_test)
rmse=np.sqrt(mse)
print("RMSE:",rmse)
print("回归系数:",linear.coef_.reshape(8,1))

RMSE: 3.563616720553335
回归系数: [[-6.34553799e-02]
 [ 5.97664148e-04]
 [ 2.20460496e-05]
 [-7.35217770e-03]
 [ 3.22538879e-04]
 [-5.86404670e-03]
 [-2.54268126e-03]
 [-2.68336854e-03]]


### 多元线性回归正规方程实现
- $Y=X\beta+\epsilon$

- 误差项:
$e=Y-X\beta$

- 损失函数：
$\displaystyle \sum_{i=1}^n{e_i^2}=e^Te$
    
- 求解:
$\frac{\partial e^Te}{\partial \beta}=-2X^TY+2X^TX\beta=0$  

- 📌 $\beta=(X^TX)^{-1}X^TY$ 📌

In [7]:
# 训练数据集
# 特征X(1600,8)，标签Y(1600,)
class LR:
    def __init__(self) -> None:
        self.coefv = None
    
    '''
    函数说明：训练
    Parameters:
        x - 数据集features
        y - 数据集labels
    Returns:
        self.coefv - 回归系数
    '''
    def fit(self,x,y):
        xMat = np.mat(x); yMat = np.mat(y).T
        xTx = xMat.T*xMat
        if np.linalg.det(xTx) == 0.0:
            print("矩阵为奇异矩阵，不能求逆")
            return
        self.coefv = xTx.I * (xMat.T*yMat)
        return self.coefv
    
    '''
    函数说明：预测
    Parameters:
        x_test - 测试集features
    Returns:
        yp - 回归预测结果
    '''    
    def predict(self,x_test):
        xtMat=np.mat(x_test)
        yp=np.array(xtMat*self.coefv)
        return yp
    

In [8]:
LRmodel=LR()
LRmodel.fit(x_train,y_train)
yp=LRmodel.predict(x_test)
mse=MSE(yp,y_test)
rmse=np.sqrt(mse)
print("RMSE:",rmse)
print("回归系数:",LRmodel.coefv)

RMSE: 15.619369247273445
回归系数: [[ 0.2136734 ]
 [-0.00836224]
 [-0.00699244]
 [ 0.01974431]
 [-0.0082733 ]
 [ 0.00714718]
 [ 0.00294814]
 [ 0.00493109]]


## 四、扩展

### 局部加权线性回归(Locally Weighted Linear Regression)
- 📌 $\beta=(X^TWX)^{-1}X^TWY$ 📌
  
  - $w(i,i)=exp(\frac{|x^{(i)}-x|^2}{-2k^2})$  
  
    LWLR使用"核"（与支持向量机中的核类似）来对附近的点赋予更高的权重。  
    核的类型可以自由选择，最常用的核就是高斯核。

In [9]:
# 训练数据集
# 特征X(1600,8)，标签Y(1600,)
class LWLR:
    def __init__(self) -> None:
        pass
    
    '''
    函数说明：训练预测
    Parameters:
        x_test - 测试集features
    Returns:
        yp - 回归预测结果
    '''    
    def fit_predict(self,x,y,x_test,k):
        '''
        函数说明：局部加权线性回归
        Parameters:
            x - 数据集features
            y - 数据集labels
            xw - 加权样本点
            k - 高斯核的k,自定义参数
        Returns:
            coefv - 回归系数
        '''
        def lwlr(x,y,xw,k=1.0):
            xMat = np.mat(x); yMat = np.mat(y).T
            n = np.shape(xMat)[0]
            weights = np.mat(np.eye((n)))                                     
            for j in range(n):                                                  
                diffMat = xw - xMat[j, :]
                weights[j, j] = np.exp(diffMat * diffMat.T/(-2.0 * k**2))
            xTx = xMat.T*(weights*xMat)
            if np.linalg.det(xTx) == 0.0:
                print("矩阵为奇异矩阵，不能求逆")
                return 
            coefv = xTx.I * (xMat.T*(weights*yMat))
            return coefv
        
        xtMat=np.mat(x_test)
        nt = np.shape(xtMat)[0]
        yp = np.zeros(nt)  
        for i in range(nt):                                                
            yp[i] = xtMat[i]*lwlr(x,y,xtMat[i],k)
        return yp
        

In [10]:
LWLRmodel=LWLR()
yp=LWLRmodel.fit_predict(x_train,y_train,x_test,100)
mse=MSE(yp,y_test)
rmse=np.sqrt(mse)
print("RMSE:",rmse)


RMSE: 8.487375720323163


### Lasso回归

- Lasso回归即我们所说的L1正则线性回归，在一般的线性回归最小化均方误差的基础上增加了一个参数$\beta$的L1范数的罚项，  
从而最小化罚项残差平方和： 

  - $min||X\beta-Y||_2^2+\lambda||\beta||_1$  


In [11]:
from sklearn.linear_model import Lasso
lasso=Lasso(alpha=0.5)
lasso.fit(x_train,y_train)
yp=lasso.predict(x_test)
mse=MSE(yp,y_test)
rmse=np.sqrt(mse)
print("RMSE:",rmse)
print("回归系数:",lasso.coef_.reshape(8,1))

RMSE: 3.562765782178886
回归系数: [[-0.06317094]
 [ 0.00051024]
 [-0.        ]
 [-0.00733419]
 [ 0.00023829]
 [-0.00586588]
 [-0.00239857]
 [-0.00267569]]


### 岭回归(Ridge Regression)
- 📌 $\beta=(X^TWX+\lambda I)^{-1}X^TWY$ 📌  

- 岭回归即我们所说的L2正则线性回归，在一般的线性回归最小化均方误差的基础上增加了一个参数$\beta$的L2范数的罚项，  
从而最小化罚项残差平方和： 

  - $min||X\beta-Y||_2^2+\lambda||\beta||_2^2$  


### 岭回归sklearn实现

In [12]:
from sklearn.linear_model import Ridge
ridge=Ridge(alpha=0.5)
ridge.fit(x_train,y_train)
yp=ridge.predict(x_test)
mse=MSE(yp,y_test)
rmse=np.sqrt(mse)
print("RMSE:",rmse)
print("回归系数:",ridge.coef_.reshape(8,1))

RMSE: 3.5636166953496056
回归系数: [[-6.34553649e-02]
 [ 5.97664006e-04]
 [ 2.20454669e-05]
 [-7.35217759e-03]
 [ 3.22538499e-04]
 [-5.86405037e-03]
 [-2.54268107e-03]
 [-2.68336874e-03]]


### 岭回归正规方程实现

In [13]:
# 训练数据集
# 特征X(1600,8)，标签Y(1600,)
class RR:
    def __init__(self) -> None:
        self.coefv = None
    
    '''
    函数说明：训练
    Parameters:
        x - 数据集features
        y - 数据集labels
        lam - 正则系数
    Returns:
        self.coefv - 回归系数
    '''
    def fit(self,x,y,lam):
        '''
        函数说明：数据标准化(-均值/方差)
        Parameters:
            x - 数据集features
            y - 数据集labels
        Returns:
            xMat,yMat - 标准化后数据集
        '''
        def normalize(x,y):
            xMat = np.mat(x); yMat = np.mat(y).T
            yMean = np.mean(yMat, axis = 0)            
            yMat = yMat - yMean                         
            xMeans = np.mean(xMat, axis = 0)         
            xVar = np.var(xMat, axis = 0)                        
            xMat = (xMat - xMeans) / xVar
            return xMat,yMat
        
        # xMat,yMat=normalize(x,y)
        xMat = np.mat(x); yMat = np.mat(y).T
        xTx = xMat.T*xMat+np.eye(np.shape(xMat)[1])*lam
        if np.linalg.det(xTx) == 0.0:
            print("矩阵为奇异矩阵，不能求逆")
            return
        self.coefv = xTx.I * (xMat.T*yMat)
        return self.coefv
    
    '''
    函数说明：预测
    Parameters:
        x_test - 测试集features
    Returns:
        yp - 回归预测结果
    '''    
    def predict(self,x_test):
        xtMat=np.mat(x_test)
        yp=np.array(xtMat*self.coefv)
        return yp

In [14]:
RRmodel=RR()
RRmodel.fit(x_train,y_train,0.5)
yp=RRmodel.predict(x_test)
mse=MSE(yp,y_test)
rmse=np.sqrt(mse)
print("RMSE:",rmse)
print("回归系数:",RRmodel.coefv)

RMSE: 15.619369249715453
回归系数: [[ 0.21367337]
 [-0.00836224]
 [-0.00699244]
 [ 0.01974431]
 [-0.0082733 ]
 [ 0.0071472 ]
 [ 0.00294814]
 [ 0.00493109]]


### 前向逐步线性回归(Forward Stepwise Linear Regression)

- 🧠核心思想：根据最小平方误差迭代寻找最佳回归
- 算法：  
  初始化回归系数、损失值L  
  迭代循环：  
  &emsp;&emsp;属性循环：  
  &emsp;&emsp;&emsp;&emsp;增减循环：  
  &emsp;&emsp;&emsp;&emsp;对属性进行更新增减步长  
  &emsp;&emsp;&emsp;&emsp;计算当前预测值  
  &emsp;&emsp;&emsp;&emsp;计算平方误差和E  
  &emsp;&emsp;&emsp;&emsp;如果E < L：  
  &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;保持修改  
  &emsp;&emsp;&emsp;&emsp;否则：  
  &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;保持不变


In [15]:
# 训练数据集
# 特征X(1600,8)，标签Y(1600,)
class FSLR:
    def __init__(self) -> None:
        self.coefv = None
    
    '''
    函数说明：训练
    Parameters:
        x - 数据集features
        y - 数据集labels
        eps - 步长
        numIt - 迭代次数
        coefInit - 初始化回归系数
    Returns:
        self.coefv - 回归系数
    '''
    def fit(self,x,y,eps,numIt,coefInit):
        xMat = np.mat(x); yMat = np.mat(y).T
        n,m = np.shape(xMat)
        coef=coefInit.copy()
        lowestError = float('inf')
        for i in range(numIt):                                                                     
            for j in range(m):                                                      
                for sign in [-1, 1]:
                    coefcopy=coef.copy()
                    coefcopy[j] += eps * sign                                      
                    yp = xMat * coefcopy                                     
                    rssE = ((yp.A-yMat.A)**2).sum()                           
                    if rssE < lowestError:                                       
                        lowestError = rssE
                        coef_max = coefcopy
                coef=coef_max.copy()
        self.coefv=coef
        return self.coefv
    
    '''
    函数说明：预测
    Parameters:
        x_test - 测试集features
    Returns:
        yp - 回归预测结果
    '''    
    def predict(self,x_test):
        xtMat=np.mat(x_test)
        yp=np.array(xtMat*self.coefv)
        return yp

In [16]:
FSLRmodel=FSLR()
coefInit=np.zeros((np.shape(x_train)[1], 1))
FSLRmodel.fit(x_train,y_train,0.0001,100000,coefInit)
yp=FSLRmodel.predict(x_test)
mse=MSE(yp,y_test)
rmse=np.sqrt(mse)
print("RMSE:",rmse)
print("回归系数:",FSLRmodel.coefv)

RMSE: 15.624929088879103
回归系数: [[ 0.2121]
 [-0.0094]
 [-0.0074]
 [ 0.0198]
 [-0.0088]
 [ 0.008 ]
 [ 0.0047]
 [ 0.0052]]


### ⭕Try Something

尝试用FSLR方法对多元线性正规方程实现和岭回归正规方程实现得到的结果进一步微调优化结果：  

- 即初始化用上述两个方法得到的结果，效果不佳，几乎没有变化😅

- ❗❗❗可忽略❗❗❗

In [17]:
FSLRmodel=FSLR()
coefInit=LRmodel.coefv
FSLRmodel.fit(x_train,y_train,0.0001,100000,coefInit)
yp=FSLRmodel.predict(x_test)
mse=MSE(yp,y_test)
rmse=np.sqrt(mse)
print("RMSE:",rmse)
print("回归系数:",FSLRmodel.coefv)

RMSE: 15.619489997579727
回归系数: [[ 0.2134734 ]
 [-0.00836224]
 [-0.00699244]
 [ 0.01974431]
 [-0.0082733 ]
 [ 0.00724718]
 [ 0.00294814]
 [ 0.00493109]]


In [18]:
FSLRmodel=FSLR()
coefInit=RRmodel.coefv
FSLRmodel.fit(x_train,y_train,0.0001,100000,coefInit)
yp=FSLRmodel.predict(x_test)
mse=MSE(yp,y_test)
rmse=np.sqrt(mse)
print("RMSE:",rmse)
print("回归系数:",FSLRmodel.coefv)

RMSE: 15.61925769851819
回归系数: [[ 0.21387337]
 [-0.00836224]
 [-0.00699244]
 [ 0.01974431]
 [-0.0082733 ]
 [ 0.0070472 ]
 [ 0.00294814]
 [ 0.00493109]]
